# Pelatihan OBSPY 2021

## REVIEW PERTEMUAN I (16 NOV 2021)

## 1- Membaca dan Menyimpan Data Waveform

- Pada pertemuan 1, kita sudah melakukan pembacaan data waveform dan Obspy memiliki kemampuan untuk membaca data dalam beberapa format 
- Salah satu format file standard yang umum digunakan adalah miniSEED.

In [ ]:
#import fungsi read dari modul obspy
from obspy import read

#membaca file mseed yang tersimpan pada direktori obspy
st1 = read()

#print variable st untuk melihat waveform
print(st1)


In [ ]:
#membaca file mseed yang tersimpan pada direktori yang sama dengan file ini.
st2 = read("./exampleData/000000000_0036EE80.mseed")

#print variable st untuk melihat waveform
print(st2)

Script di atas meminta ObsPy untuk membaca seismogram dengan menggunakan fungsi read dan menyimpannya dalam variabel st sebagai obyek Stream.

- Stream Object
Stream merupakan class object yang mengatur kumpulan Trace. Trace sendiri adalah obyek dengan data series kontinu rekaman seismik. Pada contoh di atas, file miniSEED seismogram terdiri dari 3 buah Trace yaitu dua rekaman seismogram komponen horizontal (BH1 dan BH2) serta satu rekaman seismogram komponen vertikal (BHZ). Silahkan kunjungi link ini untuk mengetahui lebih lanjut konvensi penamaan seismic channel.

- Trace Object
Obyek Stream pada dasarnya adalah type data “list” dalam bahasa pemrograman python yang artinya kumpulan data di dalamnya dapat diakses melalui indeks. 
Untuk mengakses trace rekaman channel BHZ dapat dilakukan dengan perintah sebagai berikut:

In [ ]:
tr = st1[2]
print(tr)

Obyek trace merupakan kumpulan data kontinu yang menggambarkan rekaman getaran yang terjadi di bawah permukaan bumi. Data tersebut berupa python array yang dapat diakses dengan cara:

In [ ]:
data = tr.data

#print data
print(data)

In [ ]:
#cek tipe data
type(data)

Selain memuat data rekaman seismik, obyek trace juga menyimpan informasi lain yang terdapat dalam class Stats. 
Informasi yang dimuat diantaranya berupa kode network, station, location dan channel dari data.

In [ ]:
#untuk akses stats pada trace
print(tr.stats)

Masing-masing informasi tersebut dapat diakses dan disimpan dalam variabel tersendiri untuk mempermudah jika nanti diperlukan.

In [ ]:
starttime = tr.stats.starttime
endtime = tr.stats.endtime

#print
print(starttime)
print(endtime)

## 2- Plot Waveform (Seismogram)

- Salah satu kemudahan pada modul ObsPy adalah pembuatan plot data seismogram dengan tampilan default yang telah disiapkan sehingga pengguna tidak perlu mengatur nilai x, y, ticks, judul maupun skala dari hasil plot.

- Menganut konsep Object-Oriented Programming, obyek Trace telah dilengkapi dengan public method plot yang dapat langsung dieksekusi dengan cara:

In [ ]:
#perintah untuk plot wavform (stream)
st1.plot()

In [ ]:
#perintah untuk plot wavform (trace)
tr.plot()

Fungsi plot memberikan beberapa pilihan untuk mengatur hasil plot wavform. 
Pada script berikut, plot akan diubah menjadi warna biru (color = blue), menggunakan format tick waktu AM/PM, serta pemotongan tampilan seismogram menjadi 1800 s atau 30 menit setelah starttime hingga 900 detik atau 15 menit sebelum endtime.

In [ ]:
tr.plot(color='blue', tick_format='%I:%M %p', 
        starttime=tr.stats.starttime-5, endtime=tr.stats.endtime+20)

In [ ]:
#membaca data waveform pada direktori
st = read("./exampleData/000000000_0036EE80.mseed")
tr = st[2]
print(tr)

In [ ]:
#plot secara keseluruhan
#tr.plot()

#mengatur plot
tr.plot(starttime=tr.stats.starttime, endtime=tr.stats.endtime, 
        size=(800, 250), dpi=100, color='blue', bgcolor='white', 
        tick_rotation=0, type='normal', linewidth=0.2, linestyle='-')

Dapat dilihat bahwa fungsi plot memiliki beberapa pilihan parameter yang dapat diubah seperti:
- starttime : menentukan titik awal data untuk tampilan plot
- endtime : menentukan titik akhir data untuk tampilan plot
- size : ukuran plot
- dpi : resolusi dot per inch
- color : warna grafik
- bgcolor : warna latar
- transparent : menentukan apakah latar akan transparan atau tidak
- tick_rotation : kemiringan tick dalam derajat
- type : tipe plot yang diinginkan
- linewidth : tebal garis
- linestyle : style garis
- outfile : untuk menyimpan hasil plot

Dengan mengubah nilai dari paramater / argumen yang dimiliki oleh fungsi plot maka kita dapat membuat hasil plot menjadi berbeda. Sebagai contoh:

In [ ]:
tr.plot(starttime=tr.stats.starttime+1800, endtime=tr.stats.endtime-900, 
        size=(900, 350), dpi=120, color='white', bgcolor='#000000', 
        transparent=False, tick_rotation=20, linewidth=.75, 
        linestyle=':')

## 3- Plot Waveform (Seismogram) 1 hari

In [ ]:
st = read("./exampleData/IT.ITB-J..HNE.D.2021.201")   
print(st)
st.plot(type='dayplot')

# Signal Processing

## 1- Remove Instrument Response

- Hasil rekaman seismogram tidak secara langsung merepresentasikan getaran yang terjadi di bawah permukaan bumi. 
- Rekaman seismograf tersebut merupakan gabungan (coupling) dari berbagai faktor, salah satunya adalah respon instrumen itu sendiri. 
- Setiap instrumen memiliki karakteristik masing-masing yang mana hal tersebut sangat mempengaruhi cara kerja instrumen dalam melakukan perekaman dan menghasilkan data.

Sebelum melakukan analisis lebih lanjut terhadap seismogram, langkah pertama yang harus kita lakukan adalah menghilangkan respon instrumen untuk mendapatkan representasi aktual dari getaran bawah permukaan bumi.



In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
client = Client("IRIS")
#st = read("./exampleData/IT.ITB-J..HNE.D.2021.201") 
starttime = UTCDateTime("2020-07-22T06:00:00.000")
endtime = starttime + 3600

#Gunakan fungsi get_stations()untuk mendapatkan informasi mengenai stasiun seismik.
inv = client.get_stations(network="IU", station="ANMO", location="00", 
                          channel="LHZ", level="response", starttime=starttime, 
                          endtime=endtime)
print(inv)

#st.plot()

In [ ]:
inv.plot_response(min_freq=0.001)

Gambar di atas merupakan grafik Bode yang terdiri dari dua grafik yaitu respon amplitudo dan respon fase terhadap frekuensi. Jika kita perhatikan, instrumen kanal LHZ merekam dengan tr.stats.sampling_rate = 1 sampel per detik sehingga memiliki Frekuensi Nyquist sebesar 0.5 Hz (garis putus-putus vertikal).

Plot di atas tidak menunjukkan respon instrumen semata melainkan seluruh rangkaian proses perekaman data termasuk proses konversi analog ke digital maupun tahapan filter digital yang mungkin terjadi. 

In [ ]:
st = client.get_waveforms(network="IU", station="ANMO", location="00", 
                          channel="LHZ", starttime=starttime, endtime=endtime)
st.plot()

In [ ]:
Trace.remove_response(inventory=None, output=’VEL’, water_level=60, 
                      pre_filt=None, zero_mean=True, taper=True, 
                      taper_fraction=0.05, plot=False, fig=None

- inventory : file metadata stasiun, tidak diperlukan jika sebelumnya telah melakukan attach_response terhadap data waveform
- output : Satuan output, dapat berupa "DISP", "VEL" atau "ACC"
- water_level : water level untuk proses dekonvolusi
- pre_filt : Bandpass filter pada domain frekuensi sebelum proses dekonvolusi. Berupa tuple (f1, f2, f3, f4)
- zero_mean : Mean waveform akan dikurangi pada domain waktu sebelum proses dekonvolusi
- taper : Cosine taper domain waktu sebelum dekonvolusi
- taper_fraction : Nilai cosine taper
- plot : Menampilkan plot proses koreksi respon instrumen

In [ ]:
tr = st[0]

pre_filt = [0.001, 0.005, 45, 50]
tr.remove_response(inventory=inv, pre_filt=pre_filt, output="VEL",
                   water_level=60, plot=True)

## 2- Filtering


In [ ]:
# Import modul
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
# Pilih IRIS sebagai client
client = Client("IRIS")
# Tentukan rentang waktu
starttime = UTCDateTime("2020-07-22T06:00:00.000")
endtime = starttime + 3600
# Request waveform
st = client.get_waveforms(network="IU", station="ANMO", location="00", channel="LHZ", starttime=starttime, 
                           endtime=endtime, attach_response = True)

# Koreksi respon instrumen
st.remove_response(output="VEL")
st.detrend('linear')
st.detrend('demean')
st.plot()

In [ ]:
# Import modul
import numpy as np
import matplotlib.pyplot as plt

# Menghitung rentang frequency (untuk plot sumbu x)
npts = st[0].stats.npts 
dt = st[0].stats.delta                  
fNy = 1. / (2. * dt)                     
freq = np.linspace(0, fNy, npts // 2 + 1)

# FFT untuk nilai amplitudo (sumbu y)
amp = np.fft.rfft(st[0].data)


# Plotting
plt.plot(freq, abs(amp), 'k')
plt.title('frequency-domain data \n amplitude spectrum')
plt.ylabel('amplitude')
plt.xlim(0,0.12)

In [ ]:
# Menghitung rentang frekuensi
npts = st[0].stats.npts    
dt = st[0].stats.delta               
fNy = 1. / (2. * dt)                     
freq = np.linspace(0, fNy, npts // 2 + 1)

# Rentang waktu
time = np.arange(0, npts) * dt

# Parameter lowpass                           
corners = 4
f0 = 0.04

# Proses lowpass filtering
stLP = st.copy()
stLP.filter('lowpass', freq=f0, corners=corners, zerophase=True)

# Menghitung amplitudo
LPspec = np.fft.rfft(stLP[0].data)

# ---------Plotting------------#
plt.rcParams['figure.figsize'] = 17, 4
tx1 = 500
tx2 = 3600
fx2 = 0.12

fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.ticklabel_format(style='sci', axis='y', scilimits=(-1,1))
plt.plot(time, stLP[0].data, 'k')
plt.title('time-domain data')
plt.xlim(tx1, tx2)
plt.ylabel('LOWPASS  \n amplitude [ms$^-1$]')

ax2 = fig.add_subplot(1,2,2)
plt.plot(freq, abs(LPspec), 'k')
plt.title('frequency-domain data \n amplitude spectrum')
plt.ylabel('amplitude ')
plt.xlim(0,fx2)

Jika kita perhatikan, fungsi filter pada script di atas diakses melalui object stream yang kita definisikan, yaitu stLP. Fungsi filter membutuhkan setidaknya satu argumen yakni tipe filter yang diinginkan yang dapat diikuti oleh beberapa argumen pilihan. Pada contoh di atas, kita memilih tipe filter “lowpass” dengan beberapa argumen yang dapat ditambahkan seperti:

- freq : batas nilai frekuensi
- df : sampling rate frekuensi
- corners : orde frekuensi
- zerophase : memastikan zerophase shift


## 2- Plot Spektogram

In [ ]:
#import modul yang diperlukan

from obspy.clients.fdsn import Client
from obspy import UTCDateTime

In [ ]:
client = Client("GFZ")
starttime = UTCDateTime("2020-08-18T22:23:00")
st = client.get_waveforms("GE", "MNAI", "", "BHZ", 
                          starttime, starttime + 15 * 60, attach_response=True)

In [ ]:
#Lakukan koreksi respon instrumen, kemudian plot Trace.

# remove respon instrumen
st = st.remove_response( output="VEL" )
tr = st[0]
tr.plot()


In [ ]:
#Untuk melihat spektrogram dari trace tersebut dapat dilakukan dengan cara:

tr.spectrogram()


In [ ]:
spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False, outfile=None, 
            fmt=None, axes=None, dbscale=False, mult=8.0, 
            cmap=<matplotlib.colors.LinearSegmentedColormap object at 0x43166aac>, 
            zorder=None, title=None, show=True, sphinx=False, clip=[0.0, 1.0])


In [ ]:
tr.spectrogram(log=True, dbscale=True)

## 3- Resample Data

In [ ]:
import obspy
from obspy.clients.fdsn import Client

#Read Data
client = Client("IRIS")
t = obspy.UTCDateTime("2020-07-06T22:54:47.0")  # South Java Earthquake
st = client.get_waveforms("IU", "MBWA", "00", "BHZ",
                          t - 5 * 60, t + 15 * 60)

inv = client.get_stations(network="IU", station="MBWA", location="00", channel="BHZ",
                          level="response", starttime=t - 10, endtime=t + 10)
st.plot()


In [ ]:
tr=st[0]
tr2=tr.copy() #copy file
print(tr2)
tr2.plot()

#Resample Data
tr2.resample(sampling_rate=1)
print(tr2)
tr2.plot()

## 4- Downsampling Data

In [ ]:
tr=st[0]
tr2=tr.copy() #copy file
print(tr2)
tr2.plot()

#Downsampling data 40 Hz ke 20 Hz dengan faktor 2.
tr2.decimate(factor=2, strict_length=False)
print(tr2)
tr2.plot()

## 5- Cutting Data 

In [ ]:
tr=st[0]
tr2=tr.copy() #copy file
print(tr2)

#cut the data
tr2.trim(tr2.stats.starttime + 8 * 60, tr2.stats.starttime + 10 * 60)
print(tr2)
tr2.plot()